In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [3]:
# from google.oauth2 import service_account
# from google.cloud import bigquery

# # 빅쿼리 설정
# SERVICE_ACCOUNT_FILE = "./api_key.json"  # 키 json 파일
# credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)
# project_id = "bigquery-test-408414" # 각자 프로젝트에 맞게 수정
# client = bigquery.Client(credentials=credentials, project=project_id)

# def import_bigquery_data(query):
#     query_job = client.query(query)
#     return query_job.to_dataframe()

# events = import_bigquery_data("""
#     SELECT * 
#     FROM `bigquery-public-data.thelook_ecommerce.events` 
#     WHERE created_at BETWEEN '2024-01-01' AND '2024-09-01'
# """)

# events.to_csv('event.csv', index=False)

In [4]:
order_items = pd.read_csv('order_items.csv')
orders = pd.read_csv('orders.csv')
users = pd.read_csv('users.csv')
events = pd.read_csv('event.csv')

In [5]:
# revenue 데이터 생성
using_merge_data = pd.merge(
    order_items[~order_items['status'].isin(['Cancelled','Returned'])],
    orders,
    how='inner',
    on='order_id'
)
using_merge_data['revenue'] = round(using_merge_data['num_of_item'] * using_merge_data['sale_price'])
order_data = using_merge_data[['order_id','product_id','revenue']]

---

# 2. 퍼널 차트
- 참고 링크 : https://velog.io/@suminwooo/AARRR-%EC%A0%95%EB%A6%AC

### AARRR 개념
1. Acquisition(획득)
  - 이러한 기준은 서비스에 따라 신규 방문, 앱 설치, 서비스 결제 등으로 설정 가능
  - 신규 유저 획득을 위한 적절한 채널을 확보하는 게 중요
2. Activation(활동, 활성화)
  - 획득과 마찬가지로 서비스에 따라 상이
  - 장바구니 담기, 상세 페이지 조회, 회원 가입으로 볼 수 있음
3. Retention(유지)
  - 고객 유지가 잘 되고 있느냐를 확인하는 단계
  - 재구매율, 재방문율등 서비스에 따라 상이
4. Revenue(매출)
5. Refferal(추천)
  - 마지막 단계인 만큼 퍼널에서 가장 아래에 위치
  - 추천으로 이뤄지기 어려운 만큼 다양한 방법에 대해 고민이 필요한 단계

### 아래의 예시에서 사용할 지표
1. Acquisition(획득) : 유저 유입
2. Activation(활동, 활성화) : 회원가입
3. Retention(유지) : 재방문
4. Revenue(매출) : 구매
5. Refferal(추천) : 추천 

참고사항 : 예시는 한달 데이터만 활용(2024년 7월)

In [6]:
# 2024년 7월 가입 유저
july_usrs_lst = users[
    (users['created_at']>='2024-07-01') & 
    (users['created_at']<'2024-08-01')
]['id'].tolist()

In [7]:
events['user_id'] = events['user_id'].fillna(-1)

using_events = events[
    (events['created_at']>='2024-07-01') & 
    (events['created_at']<'2024-08-01') & 
    (
        (events['user_id']==-1) | (events['user_id'].isin(july_usrs_lst))
    )
]

### 1. Acquisition
- 유저 유입 계산
- 편의상 해당일 접속 user_id + user_id가 없는 경우엔 session_id 수

In [8]:
user_id_lst = list(set(using_events[using_events['user_id']!=-1]['user_id'].astype(int)))

# 이후 값 통일을 위해 정수형 변형후 문자형으로
user_id_lst = [str(int(i)) for i in user_id_lst]

In [9]:
# 비회원인 경우에는 유저의 session_id을 뺀 값을 활용
user_session_lst = set(using_events[using_events['user_id'].isin(user_id_lst)]['session_id'])
not_user_session_lst = set(using_events[using_events['user_id']==-1]['session_id'])

not_usr_lst = list(not_user_session_lst - user_session_lst)

In [10]:
acquisition = pd.DataFrame([user_id_lst + not_usr_lst], index=['user_id']).T
acquisition['type'] = 1

In [11]:
acquisition.head()

user_id  type
0   12290     1
1   57348     1
2   73735     1
3   38921     1
4   49163     1

### 2. Activation(활동, 활성화) : 회원가입
- 1번에서 편의상 회원은 구해두었으므로, 그대로 활용

In [12]:
activation = pd.DataFrame(user_id_lst, columns=['user_id'])
activation['type'] = 2

In [13]:
activation.head()

user_id  type
0   12290     2
1   57348     2
2   73735     2
3   38921     2
4   49163     2

### 3. Retention
- 홈 방문 및 장바구니 기준으로 설정

In [14]:
set(using_events['event_type'])

{'cancel', 'cart', 'department', 'home', 'product', 'purchase'}

In [15]:
value_counts = using_events[
    (using_events['event_type'].isin(['home','cart'])) & 
    (using_events['user_id']!=-1)
]['user_id'].value_counts()
revisit_usr_lst = list(value_counts[value_counts >= 2].index)

# 이후 값 통일을 위해 정수형 변형후 문자형으로
revisit_usr_lst = [str(int(i)) for i in revisit_usr_lst]

In [16]:
retention = pd.DataFrame(revisit_usr_lst, columns=['user_id'])
retention['type'] = 3

In [17]:
retention.head()

user_id  type
0   52347     3
1   23907     3
2   30844     3
3   25135     3
4   60640     3

### 4. Revenue


In [18]:
order_usr_lst = list(set(using_events[using_events['event_type']=='purchase']['user_id']))

# 이후 값 통일을 위해 정수형 변형후 문자형으로
order_usr_lst = [str(int(i)) for i in order_usr_lst]

In [19]:
revenue = pd.DataFrame(order_usr_lst, columns=['user_id'])
revenue['type'] = 4

In [20]:
revenue.head()

user_id  type
0   12290     4
1   57348     4
2   73735     4
3   38921     4
4   49163     4

### 5. Refferal
- 생략

## 퍼널 그래프 생성하기

### plotly 문서 : https://plotly.com/graphing-libraries/

- Plotly는 데이터 시각화를 위한 라이브러리
- 인터랙티브한 그래프: Plotly의 그래프는 마우스 오버, 줌, 팬 등 다양한 상호작용을 지원하여 데이터를 더 깊이 탐색
- 다양한 차트 유형: Plotly는 선 그래프, 막대 그래프, 파이 차트, 히스토그램, 산점도 등 여러 가지 차트 유형을 지원
- 웹 기반 시각화: Plotly는 HTML, JavaScript와 잘 통합되어 있어 웹 페이지에서 직접 시각화를 사용 가능

In [1]:
import plotly.graph_objects as go

# 데이터 준비
x = [1, 2, 3, 4, 5]
y = [10, 11, 12, 13, 14]

# 그래프 생성
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='lines+markers'))

# 그래프 제목과 축 레이블 설정
fig.update_layout(
    xaxis_title='X axis',
    yaxis_title='Y axis'
)

# 그래프 출력
fig.show()

#### 퍼널 차트 생성을 위한 plotly 문서 : https://plotly.com/python/funnel-charts/

In [21]:
from plotly import graph_objects as go

In [22]:
# 퍼널 데이터 생성
stage = ['Acquisition','Activation','Retention','Revenue']
usr_cnt = [len(acquisition), len(activation), len(retention), len(revenue)]

In [23]:
usr_cnt

[8259, 610, 609, 598]

In [24]:
fig = go.Figure(go.Funnel(
    y = stage,
    x = usr_cnt,
    textposition = "inside",
    textinfo = "value+percent initial",
    opacity = 0.65, marker = {"color": ["deepskyblue", "lightsalmon", "tan", "teal", "silver"],
    "line": {"width": [4, 2, 2, 3, 1, 1], "color": ["wheat", "wheat", "blue", "wheat", "wheat"]}},
    connector = {"line": {"color": "royalblue", "dash": "dot", "width": 3}})
    )
fig.show()